# Chapter 8 - Exercises

## Set Up

### Packages

In [1]:
import os

import arviz as az
import graphviz as gr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc as pm
import seaborn as sns
from scipy import stats
from scipy.interpolate import BSpline
from scipy.special import expit
from sklearn.preprocessing import StandardScaler

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


### Defaults

In [2]:
# seaborn defaults
sns.set(
    style="whitegrid",
    font_scale=1.2,
    rc={
        "axes.edgecolor": "0",
        "axes.grid.which": "both",
        "axes.labelcolor": "0",
        "axes.spines.right": False,
        "axes.spines.top": False,
        "xtick.bottom": True,
        "ytick.left": True,
    },
)

colors = sns.color_palette()

### Constants

In [3]:
DATA_DIR = "../data"
HOWELL_FILE = "howell.csv"
CHERRY_BLOSSOMS_FILE = "cherry_blossoms.csv"
WAFFLE_DIVORCE_FILE = "waffle_divorce.csv"
MILK_FILE = "milk.csv"
LDS_FILE = "lds_by_state.csv"
CHIMPANZEES_FILE = "chimpanzees.csv"
ADMISSIONS_FILE = "ucbadmit.csv"
KLINE_FILE = "kline.csv"
REEDFROGS_FILE = "reedfrogs.csv"
CARS_FILE = "cars.csv"
RUGGED_FILE = "rugged.csv"
TULIPS_FILE = "tulips.csv"
LAFFER_FILE = "Laffer.csv"
FOXES_FILE = "foxes.csv"

RANDOM_SEED = 42

In [4]:
def load_data(file_name, data_dir=DATA_DIR, **kwargs):
    path = os.path.join(data_dir, file_name)
    return pd.read_csv(path, **kwargs)

## Easy

### 8E1

For each of the causal relationships below, name a hypothetical third variable that would lead to an interaction effect.
1. Bread dough rises because of yeast.
2. Education leads to higher income.
3. Gasoline makes a car go.

----

1. Yeast will only make bread dough rise in the presence of heat, so heat leads to an interaction effect.
2. The effect of education on income also depends on the opportunities available, so I would expect family wealth to lead to an interaction effect. The specific subject for higher education would also lead to an interaction effect.
3. The quantity of oxygen would be an example of an interaction effect.

### 8E2

Which of the following explanations invokes an interaction?
1. Caramelizing onions requires cooking over low heat and making sure the onions do not dry out.
2. A car will go faster when it has more cylinders or when it has a better fuel injector.
3. Most people acquire their political beliefs from their parents, unless they get them instead from their friends.
4. Intelligent animal species tend to be either highly social or have manipulative appendages (hands, tentacles, etc.)

----

1. This invokes an interaction between heat and moisture: the values don't determine caramelization independently - it is the combination that produces the effect.
2. This doesn't invoke an interaction because there's nothing to suggest that number of cylinders and the quality of the fuel injector aren't independent in their effect on speed.
3. This invokes an interaction between parents' and friends' political beliefs because the effect of parents' political beliefs is dependent on friends' political beliefs.
4. This doesn't invoke an interaction because there's no indication that the effects of being social and having appendages have anything to do with each other.

### 8E3

For each of the explanations in **8E2**, write a linear model that expresses the stated relationship.

---

1. $\text{caramelization} = \alpha + \beta_h \cdot \text{heat} + \beta_m \cdot \text{moisture} + \beta_{mh} \cdot \text{heat} \cdot \text{moisture}$.
2. $\text{speed} = \alpha + \beta_c \cdot (\text{num cylinders}) + \beta_f \cdot (\text{fuel injector quality})$.
3. $\text{political beliefs} = \alpha + \beta_p \cdot (\text{parents' beliefs}) + \beta_f \cdot (\text{friends' beliefs}) + \beta_{pf} \cdot (\text{parents' beliefs}) \cdot (\text{friends' beliefs})$.
4. $\text{intelligence} = \alpha + \beta_s \cdot \text{socialness} + \beta_a \cdot (\text{has manipulative appendages})$.